# Palabras Relacionadas - Dataset

En este notebook se explica en detalle el dataset y el procesamiento que se requirió para dejarlo listo para la siguiente etapa del proyecto.

En primer lugar, se realizaron algunas importaciones, se configuró el directorio de trabajo y se definió una función de utilidad.

In [ ]:
!pip install pypdf

In [1]:
from pypdf import PdfReader
import os
from os import listdir
from os.path import isfile, join
import pathlib

# Seteo el path al root del proyecto
dev_folder = 'dev'
folders = os.getcwd().split('/')
if (len(folders) == 1):
    folders = folders[0].split('\\')
if(folders[-1] == dev_folder):
    os.chdir('../')
print(os.getcwd())

c:\Users\saoga\OneDrive\Escritorio\Repos\TPI-RNP-Palabras-Relacionadas-ISI


In [30]:
def get_filenames(path):
    return [os.path.splitext(f)[0] for f in listdir(path) if isfile(join(path, f))]

Luego, establecimos los subdirectorios con los que se trabajó.

In [29]:
raw_path = "./data/raw"
plain_path = "./data/plain"
tokens_clean_path = "./data/tokens_clean"
tokens_full_path = "./data/tokens_full"
vocabularies_path = "./data/vocabularies"
tokens_concepts_path = "./data/tokens_concepts"
datasets_path = "./data/datasets"
models_path = "./data/models"

A continuación, se buscó y seleccionó múltiples textos en formato PDF pertenencientes a distintas materias de la carrera Ingeniería en Sistemas de Información, Universidad Tecnológica Nacional Facultad Regional Mendoza.

Estos, fueron cargados en la carpeta /data/raw.

Se siguió como convención para los nombres el número del año de la materia, seguido de un guión, una abreviatura del nombre de la materia, otro guión y el nombre original del material.

Utilizando algunas librerías de python, se convirtió cada archivo PDF en un archivo txt con su contenido, en /data/plain.

In [39]:
def convert_pdf_to_plain(filename):
    reader = PdfReader(filename)
    content = ""
    for p in reader.pages:
        content += p.extract_text(
            extraction_mode="plain",
            layout_mode_space_vertically=False)
        
    return content

In [40]:
def raw_to_plain(raw_path, plain_path, converter):

    metrics = {}

    raw_files = get_filenames(raw_path)

    pathlib.Path(plain_path).mkdir(parents=True, exist_ok=True) 
    
    for f in raw_files:
        print("\033[94mConvirtiendo archivo: " + f + "\033[0m")

        sf = f.split("-")
        anio = sf[0].strip()
        materia = sf[1].strip()
        if (not anio in metrics):
            metrics[anio] = {}

        if (not materia in metrics[anio]):
            metrics[anio][materia] = 0

        metrics[anio][materia] += os.path.getsize(raw_path + "/" + f + ".pdf")

        content = converter(raw_path + "/" + f + ".pdf")
        
        with open(plain_path + "/" + f + ".txt", "ab") as t:
            t.write(content.encode("utf-8"))
    
    return metrics

In [41]:
raw_to_plain_metrics = raw_to_plain(raw_path, plain_path, convert_pdf_to_plain)

print("\033[92m")
for a, materias in raw_to_plain_metrics.items():
    print("Año " + a + ":")
    for m, tamano in materias.items():
        print("\tMateria: " + m + " - " + str(round(tamano/1000000,2)) +"MB")
print("\033[0m")

Convirtiendo archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Convirtiendo archivo: 1 - AyED - cpp según yo pero en pedo
Convirtiendo archivo: 1 - AyED - cpp según yo
Convirtiendo archivo: 1 - AyED - Unidad3 (7929)
Convirtiendo archivo: 1 - AyED - Unidad4 (7930)
Convirtiendo archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Convirtiendo archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)


KeyboardInterrupt: 

A continuación, se tokenizó los archivos planos txt, generando dos nuevos archivos txt nuevos por cada archivo plano donde cada línea es un token. El archivo "full" contiene todos los tokens, mientras que el "clean" solo contiene los que se consideran potencialmente relevantes para detectar conceptos.

Por ejemplo, no se considera limpios a los tokens correspondientes a signos de puntuación, espacios o palabras muy frecuentes en el lenguaje.

Esto se logró gracias a Spacy.

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [32]:
import spacy

esp = spacy.load("es_core_news_sm")

In [31]:
class Token():
    def __init__(self, text, is_clean):
        self.text = text
        self.is_clean = is_clean

In [36]:
def tokenizer_esp_spacy(txt):
    def is_clean_token(token):
        return not (
            token.is_punct or
            token.is_space or
            token.is_stop or
            len(token) == 1)
    
    tokens = esp.tokenizer(txt)

    ret = []

    for token in tokens:
        ret.append(Token(token.text, is_clean_token(token)))

    return ret

In [37]:
def tokenize(plain_path, tokens_full_path, tokens_clean_path, tokenizer):
    plain_files = get_filenames(plain_path)

    pathlib.Path(tokens_full_path).mkdir(parents=True, exist_ok=True) 
    pathlib.Path(tokens_clean_path).mkdir(parents=True, exist_ok=True) 

    for f in plain_files:
        print("\033[94mTokenizando archivo: " + f + "\033[0m")

        with open(plain_path + "/" + f + ".txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = tokenizer(txt)
            with open(tokens_clean_path + "/" + f + ".txt", "wb") as tcf:
                with open(tokens_full_path + "/" + f + ".txt", "wb") as tff:
                    for token in tokens:
                        enc_token = (token.text + "\n").encode("utf-8")
                        tff.write(enc_token)
                        if (token.is_clean):
                            tcf.write(enc_token)

In [38]:
tokenize(plain_path, tokens_full_path, tokens_clean_path, tokenizer_esp_spacy)

Tokenizando archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Tokenizando archivo: 1 - AyED - cpp según yo pero en pedo
Tokenizando archivo: 1 - AyED - cpp según yo
Tokenizando archivo: 1 - AyED - Unidad3 (7929)
Tokenizando archivo: 1 - AyED - Unidad4 (7930)
Tokenizando archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Tokenizando archivo: 1 - SyO - 01 Evolucion de las estructuras
Tokenizando archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Tokenizando archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Tokenizando archivo: 1 - SyO - 04 gestion-por-procesos
Tokenizando archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Tokenizando archivo: 1 - SyO - 1)La Información en la Empresa
Tokenizando archivo: 1 - SyO - 2)Recopilacion de la informacion
Tokenizando archivo: 1 - SyO - Analisis FODA
Tokenizando archiv

KeyboardInterrupt: 

Una vez que se tienen los archivos con los tokens, deseamos detectar conceptos dentro de ellos. Estos conceptos formarán luego nuestro vocabulario.

Se utilizará un algoritmo de ventana deslizante para esto.

La función `get_candidate_concepts` analiza los tokens "clean" de cada archivo original, detectando y contando todas las secuencias de estos de longitud entre 1 y el tamaño de la ventana. A estas secuencias se les llama "conceptos candidatos".

Por otro lado, `make_vocabs` toma los conceptos candidatos y, por cada longitud de estos (del 1 al tamaño de la ventana), calcula la frecuencia promedio. Esta frecuencia promedio permite filtrar los conceptos candidatos, tomando como "conceptos definitivos" aquellos cuya frecuencia se encuentre entre ciertos valores, obtenidos al multiplicar por la frecuencia promedio.

Los conceptos definitivos pasan a formar el vocabulario.

In [44]:
import re

def get_candidate_concepts(tokens_clean_path, window_size):

    tokens_clean_files = get_filenames(tokens_clean_path)

    candidates = {}

    banned_tokens = []

    with open("./data/banned.txt", "rb") as bf:
        banned_tokens = bf.read().decode("utf-8").split("\r\n")

    for f in tokens_clean_files:
        print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

        tokens = []

        with open(tokens_clean_path + "/" + f + ".txt", "rb") as tf:
            tokens = tf.read().decode("utf-8").split("\r\n")

        for i in range(len(tokens) - window_size):
            window = tokens[i:i+window_size]
            
            arrays = [window[0:r] for r in range(1,window_size+1)]

            for arr in arrays:
                arr = [s.lower() for s in arr]

                if arr[-1] in banned_tokens:
                    break

                if any(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)|[0-9]{1,2}:[0-9]{1,2}|[0-9]{1,2}/[0-9]{1,2}/[0-9]{2,4}', s) for s in arr):
                    continue

                if len(arr) != len(set(arr)):
                    continue
                
                t = tuple(arr)
                if (not t in candidates):
                    candidates[t] = 0
                candidates[t] += 1
    return candidates


def make_vocabs(vocabularies_path, candidates, window_size, freq_ranges):
    
    pathlib.Path(vocabularies_path).mkdir(parents=True, exist_ok=True) 

    base = len(get_filenames(vocabularies_path))

    candidates_count = [0 for ws in range(window_size)]
    candidates_freqs_acum = [0 for ws in range(window_size)]
    for tokens, freq in candidates.items():
        candidates_count[len(tokens)-1] += 1
        candidates_freqs_acum[len(tokens)-1] += freq

    avg_candidates_freq = [candidates_freqs_acum[i] / candidates_count[i] for i in range(len(candidates_count))]


    for i, freq_range in enumerate(freq_ranges):
        print("\033[94mGenerando vocabulario: " + str(base + i) + " (" + str(freq_range) + ")\033[0m")
        vocabulary = []

        for tokens, freq in candidates.items():
            c = len(tokens)
            use_nth = c <= len(freq_range)
            ix = c-1 if use_nth else -1

            min_freq = freq_range[ix][0]
            max_freq = freq_range[ix][1]

            freq_rel_avg = freq / avg_candidates_freq[c-1]

            if (freq_rel_avg >= min_freq and freq_rel_avg <= max_freq):
                vocabulary.append(tokens)

        with open(vocabularies_path + "/vocab_" + str(base + i) + ".txt", "wb") as cf:
            for concept in vocabulary:
                cf.write((str(concept) + "\n").encode("utf-8"))

In [45]:
window_size = 4

candidates = get_candidate_concepts(tokens_clean_path, window_size)


Detectando conceptos en archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Detectando conceptos en archivo: 1 - AyED - cpp según yo pero en pedo
Detectando conceptos en archivo: 1 - AyED - cpp según yo
Detectando conceptos en archivo: 1 - AyED - Unidad3 (7929)
Detectando conceptos en archivo: 1 - AyED - Unidad4 (7930)
Detectando conceptos en archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Detectando conceptos en archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Detectando conceptos en archivo: 1 - SyO - 01 Evolucion de las estructuras
Detectando conceptos en archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Detectando conceptos en archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Detectando conceptos en archivo: 1 - SyO - 04 gestion-por-procesos
Detectando conceptos en archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Detectando conceptos en archivo: 1 - SyO - 1)

Para armar el vocabulario, se deben armar los rangos de frecuencias.

Cada rango de frecuencias se convertirá en un vocabulario.

Un rango de frecuencias es una secuencia de duplas, donde la enésima dupla contiene los factores para las frecuencias mínimas y máximas para los conceptos de longitud n.

Los factores para las frecuencias mínimas y máximas son multiplicados por la frecuencia promedio de los conceptos de la longitud correspondiente para determinar el mínimo y máximo.

A modo de ejemplo:
* Si se tiene un concepto candidato de dos tokens (longitud 2)
* La segunda dupla en un rango de frecuencias es (50, 100)
* En promedio, los conceptos de longitud 2 tienen frecuencia (absoluta) de 3 (es decir, en promedio se encontró tres veces a los conceptos de dos tokens)

Entonces el concepto candidato se volverá definitivo si y solo sí su frecuencia se encuentra entre 3 * 50 = 150 y 3 * 100 = 300.

Si bien los rangos de frecuencias pueden ser armados por prueba y error, al analizar un poco más a fondo se puede llegar a la conclusión de que los conceptos candidatos de n+1 tokens contienen 2 tokens de longitud n.

Como ejemplo, el concepto candidato (protocolo, tcp, ip) (de longitud 3) contiene a estos dos conceptos candidatos de longitud 2: (protocolo, tcp) y (tcp, ip).

En base a esta observación, se concluye que es conveniente, para facilitar el proceso de descubrimiento empírico del rango de frecuencias, dejar fija la relación entre las duplas correspondientes a conceptos candidatos de diferentes longitudes, dividiendo siempre el rango a la mitad.

In [ ]:
div_by_2 = lambda start_min, start_max: [(start_min*(4-i), start_max*(4-i)) for i in range(0,3)]

freq_ranges = [
    #[(67, 134),(63, 318), (44, 268), (1.5, 1.8)]
    #[(126, 636),(64, 318), (32, 159), (16, 80)]
    div_by_2(16,60)
]

make_vocabs(vocabularies_path, candidates, window_size, freq_ranges)

Generando vocabulario: 8 ([(64, 160), (48, 120), (32, 80)])
Generando vocabulario: 9 ([(64, 200), (48, 150), (32, 100)])
Generando vocabulario: 10 ([(64, 240), (48, 180), (32, 120)])


En este punto, ya hemos seleccionado conjuntos de tokens que suelen aparecer cerca.

Estos conjuntos serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo con los tokens completos (limpios o no) mediante una ventana deslizante de un tamaño proporcional al utilizado para detectar conceptos (no igual, dado que en la detección de conceptos solo había tokens limpios).

Los resultados de este proceso son escritos a archivos dentro de /data/tokens_concepts. Estos archivos consisten en secuencias de números enteros. Los positivos (o 0) corresponden al índice de un concepto en el vocabulario, mientras que los negativos indican la cantidad de tokens no reconocidos (\<unk\>) entre medio. Esto se permite ahorrar mucho espacio y tiempo de procesamiento.

In [28]:
import ast

def get_vocabulary(vocabularies_path, vocabulary):
    with open(vocabularies_path + "/" + vocabulary + ".txt", "rb") as cf:
        lines = cf.read().decode("utf-8").split("\n")[:-1]
        return [ast.literal_eval(l) for l in lines]

In [ ]:
vocabularies_path = "./data/vocabularies"
selected_vocabulary = "vocab_1"

vocabulary = get_vocabulary(vocabularies_path, selected_vocabulary)

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:441


In [150]:
import math

def tokenize_by_concepts(tokens_full_path, tokens_concepts_path, vocabulary, window_size, window_size_extension_factor = 3):

    metrics = {}

    window_size_large = math.floor(window_size * window_size_extension_factor)

    pathlib.Path(tokens_concepts_path).mkdir(parents=True, exist_ok=True) 

    tokens_full_files = get_filenames(tokens_full_path)

    for f in tokens_full_files:
        print("\033[94mTokenizando por conceptos: " + f + "\033[0m")
        metrics[f] = {}
        found_concepts = 0

        recent_concepts = {}

        with open(tokens_full_path + "/" + f + ".txt", "rb") as pf:
            tokens = pf.read().decode("utf-8").split("\n")
            tokens = [token.lower() for token in tokens]

            metrics[f]["tokens"] = len(tokens)

            with open(tokens_concepts_path + "/" + f + ".txt", "wb") as tnf:

                unks = 0

                for i in range(len(tokens) - window_size_large):
                    window = tokens[i:i+window_size_large]
                    for k, v in recent_concepts.items():
                        if v > 0:
                            recent_concepts[k] -= 1

                    unks += 1
                
                    for ix, concept in enumerate(vocabulary):
                        if (ix in recent_concepts and recent_concepts[ix] > 0):
                            continue

                        curr_word_ix = 0
                        curr_word = concept[curr_word_ix]

                        found = False

                        for token in window:
                            if(token == curr_word):
                                curr_word_ix += 1
                                if (len(concept) <= curr_word_ix):
                                    found = True
                                    break
                                curr_word = concept[curr_word_ix]

                        if found:
                            if unks > 0:
                                tnf.write(("-" + str(unks) + " ").encode("utf-8"))
                                unks = 0
                            tnf.write((str(ix) + " ").encode("utf-8"))
                            recent_concepts[ix] = window_size_large
                            found_concepts += 1

                        
        metrics[f]["concepts"] = found_concepts

    return metrics

In [ ]:
window_size_extension_factor = 3

tokenize_by_concepts_metrics = tokenize_by_concepts(tokens_full_path, tokens_concepts_path, vocabulary, window_size, window_size_extension_factor)

Tokenizando por conceptos: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Tokenizando por conceptos: 1 - AyED - cpp según yo pero en pedo
Tokenizando por conceptos: 1 - AyED - cpp según yo
Tokenizando por conceptos: 1 - AyED - Unidad3 (7929)
Tokenizando por conceptos: 1 - AyED - Unidad4 (7930)
Tokenizando por conceptos: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando por conceptos: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Tokenizando por conceptos: 1 - SyO - 01 Evolucion de las estructuras
Tokenizando por conceptos: 1 - SyO - 02 Gestion por procesos UNCuyo
Tokenizando por conceptos: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Tokenizando por conceptos: 1 - SyO - 04 gestion-por-procesos
Tokenizando por conceptos: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Tokenizando por conceptos: 1 - SyO - 1)La Información en la Empresa
Tokenizando por conceptos: 1 - SyO - 2)Recopilaci

In [152]:
print("\033[92m")
for archivo, item in tokenize_by_concepts_metrics.items():
    print("Archivo " + archivo + ":")
    for nombre, valor in item.items():
        print("\t" + nombre + ": " + str(valor))
print("\033[0m")


Archivo 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321:
	tokens: 94709
	concepts: 3867
Archivo 1 - AyED - cpp según yo pero en pedo:
	tokens: 2658
	concepts: 68
Archivo 1 - AyED - cpp según yo:
	tokens: 2554
	concepts: 75
Archivo 1 - AyED - Unidad3 (7929):
	tokens: 5728
	concepts: 175
Archivo 1 - AyED - Unidad4 (7930):
	tokens: 2146
	concepts: 116
Archivo 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928):
	tokens: 3520
	concepts: 128
Archivo 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org):
	tokens: 198363
	concepts: 4064
Archivo 1 - SyO - 01 Evolucion de las estructuras:
	tokens: 4413
	concepts: 177
Archivo 1 - SyO - 02 Gestion por procesos UNCuyo:
	tokens: 9477
	concepts: 306
Archivo 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2:
	tokens: 12599
	concepts: 380
Archivo 1 - SyO - 04 gestion-por-procesos:
	tokens: 4530
	concepts: 175
Archivo 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q:
	tokens: 

Teniendo ya las secuencias de números, se busca detectar qué conceptos se encuentran relacionados entre sí; esto es, se encuentran cerca en el texto.

Para poder armar el dataset mediante Skip-Gram, se requieren tres elementos:
* Concepto central
* Conceptos en el contexto
* Ejemplos negativos de conceptos (que no se encuentran en el centro).

El objetivo de los ejemplos negativos es evitar que la red neuronal que se entrenará piense que siempre todos los conceptos están en el contexto de otros.

La generación de los ejemplos negativos será realizado mediante un muestreo, para lo cual primero se debe calcular la frecuencia relativa de cada token:

In [153]:
def get_tokens_concepts_freqs(tokens_concepts_path):
    freq_abs = {}

    tokens_concepts_files = get_filenames(tokens_concepts_path)

    for file in tokens_concepts_files:
        with open(f"{tokens_concepts_path}/{file}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")

            nums = txt.split(" ") # Lista con cada número en el archivo
            for num in nums:
                if (num == "" or num[0] == "-"):
                    continue

                token_ix = int(num)
                if not token_ix in freq_abs:
                    freq_abs[token_ix] = 0

                freq_abs[token_ix] += 1

    total_tokens = 0
    for token, freq in freq_abs.items():
        total_tokens += freq

    freq_rel = {}
    for token, freq in freq_abs.items():
        freq_rel[token] = freq/total_tokens

    return freq_abs, freq_rel
        

A su vez, se definirá una clase auxiliar:

In [154]:
import random

class RandomGenerator:
  """Randomly draw among {1, ..., n} according to n sampling weights."""
  def __init__(self, sampling_weights):
    # Exclude
    self.population = list(range(1, len(sampling_weights) + 1))
    self.sampling_weights = sampling_weights
    self.candidates = []
    self.i = 0

  def draw(self):
    if self.i == len(self.candidates):
      # Cache `k` random sampling results
      self.candidates = random.choices(
          self.population, self.sampling_weights, k=10000)
      self.i = 0
    self.i += 1
    return self.candidates[self.i - 1]

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

In [ ]:
!pip install torch

In [ ]:
import torch
import pickle

class ISIDataset(torch.utils.data.Dataset):
    def __init__(self, load_file):
        with open(load_file, "rb") as lf:
            self.data = pickle.load(lf)
    
    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

La clase del dataset definida, sin embargo, es solo un wrapper que carga el contenido de un archivo. Estos archivos son generados mediante el algoritmo `make_datasets`. Esta función tiene la capacidad de generar velozmente múltiples datasets variando el tamaño de la ventana.

Para un único tamaño de ventana, el algoritmo es el siguiente:
1. Por cada archivo, se itera para centrarse en cada concepto.
2. Se analiza hacia el lado izquierdo del concepto para detectar conceptos en el contexto, hasta agotar la mitad del tamaño de la ventana.
3. Se analiza hacia el lado derecho del concepto, de forma análoga al izquierdo.
4. Se generan ejemplos negativos (garantizado que sean distintos a los conceptos del contexto). La cantidad de ejemplos negativos es K veces la de conceptos en el contexto.
5. Se almacenan los centros, sus contextos y ejemplos negativos en un diccionario, que a su vez es guardado en un archivo.

Si se genera más de un dataset con tamaños de ventana distintos, en primer lugar se ordenan estos tamaños de menor a mayor, y luego cambian los pasos 2. y 3.:
1. Se analiza hacia el lado izquierdo del concepto, por el primer tamaño de ventana (el más pequeño)
2. Se analiza hacia el lado izquierdo, desde el primer tamaño de ventana hasta el segundo. El proceso se repite hasta haber analizado todos los tamaños.
3. Se analiza hacia el lado derecho, de manera análoga.
4. Dado que el contexto de la ventana n siempre contiene al de la ventana n-1, se los une recursivamente.

In [175]:

def make_datasets(tokens_concepts_path, datasets_path, vocabulary, window_sizes, K):

    data = [[] for i in range(0, len(window_sizes))]

    _, freq_rel = get_tokens_concepts_freqs(tokens_concepts_path)

    sampling_weights = [freq_rel[concept]**0.75 if concept in freq_rel else 0 for concept in range(0, len(vocabulary))]
    generator = RandomGenerator(sampling_weights)

    pathlib.Path(datasets_path).mkdir(parents=True, exist_ok=True) 
    
    tokens_concepts_files = get_filenames(tokens_concepts_path)
    window_sizes.sort()

    for file in tokens_concepts_files:
        print("\033[94mArmando dataset con: " + file + "\033[0m")

        with open(f"{tokens_concepts_path}/{file}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")

            nums = txt.split(" ") # Lista con cada número en el archivo

            for ix, num in enumerate(nums):
                if (num == "" or num[0] == "-"):
                    continue

                token_ix = int(num)

                context = []
                
                curr_ws_ix = 0

                c = 0
                i = 1
                
                while curr_ws_ix < len(window_sizes): # Buscar conceptos hacia atrás
                    curr_ws = window_sizes[curr_ws_ix]
                    curr_wr = curr_ws // 2
                    c += curr_wr

                    context.append([])

                    while c >= 0: 
                        if (ix - i < 0): # Si se acabó el archivo, dejar de buscar
                            break
                        
                        val = nums[ix - i]
                        if (val == ""): 
                            val = "-0"
                        if (val[0] == "-"):
                            val = val.lstrip("-")
                            unks = int(val)
                            c = c - unks
                        else:
                            concept = int(val)
                            context[curr_ws_ix].append(concept)
                            c = c - 1
                        i = i + 1

                    curr_ws_ix += 1

                curr_ws_ix = 0

                c = 0
                i = 1
                l = len(nums)

                while curr_ws_ix < len(window_sizes): # Buscar conceptos hacia adelante
                    curr_ws = window_sizes[curr_ws_ix]
                    curr_wr = curr_ws // 2
                    c += curr_wr

                    while c >= 0: 
                        if (ix + i < l): # Si se acabó el archivo, dejar de buscar
                            break
                        
                        val = nums[ix + i]
                        if (val == ""): 
                            val = "-0"
                        if (val[0] == "-"):
                            val = val.lstrip("-")
                            unks = int(val)
                            c = c - unks
                        else:
                            concept = int(val)
                            context[curr_ws_ix].append(concept)
                            c = c - 1
                        i = i + 1

                    curr_ws_ix += 1
                
                # Juntar contextos de tamaños de ventana más grandes con otros más pequeños
                context_aux = []
                context_acc = []

                for sub_ctx in context:
                    context_acc += sub_ctx
                    context_aux.append(context_acc[:])
                
                context = context_aux

                for ctx_ix, ctx in enumerate(context):
                    negatives = []
                    
                    while len(negatives) < len(ctx) * K:
                        neg = generator.draw()
                        if neg not in context:
                            negatives.append(neg)

                    if (len(ctx) > 0):
                        data[ctx_ix].append({
                            "center": token_ix,
                            "context": ctx,
                            "negatives": negatives
                        })
    
    for ws_ix, ws in enumerate(window_sizes):
        with open(datasets_path + "/dataset-" + str(ws) + ".pkl", "wb") as lf:
            pickle.dump(data[ws_ix], lf)

Para poder probar distintas alternativas, se generó distintos datasets (almacenados en /data/datasets), con distintos tamaños de ventana y cantidad de ejemplos negativos (K).

In [ ]:
full_window_sizes = [50, 100, 200, 300]
K = 5

make_datasets(tokens_concepts_path, datasets_path, vocabulary, full_window_sizes, K)

Armando dataset con: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Armando dataset con: 1 - AyED - cpp según yo pero en pedo
Armando dataset con: 1 - AyED - cpp según yo
Armando dataset con: 1 - AyED - Unidad3 (7929)
Armando dataset con: 1 - AyED - Unidad4 (7930)
Armando dataset con: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Armando dataset con: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Armando dataset con: 1 - SyO - 01 Evolucion de las estructuras
Armando dataset con: 1 - SyO - 02 Gestion por procesos UNCuyo
Armando dataset con: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Armando dataset con: 1 - SyO - 04 gestion-por-procesos
Armando dataset con: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Armando dataset con: 1 - SyO - 1)La Información en la Empresa
Armando dataset con: 1 - SyO - 2)Recopilacion de la informacion
Armando dataset con: 1 - SyO - Analisis FODA
Armando dataset co

La siguiente celda muestra un ejemplo de cómo cargar uno de los datasets generados.

In [177]:
dataset = ISIDataset(datasets_path + "/dataset-100.pkl")

print(len(dataset))

241310


A partir de este Dataset se generará un DataLoader.

Un requisito que debe cumplir es que se modifique los lotes para que estos tengan la misma longitud (cosa que no pasa debido a las diferentes cantidades de conceptos en el contexto de cada concepto central y, por ende de ejemplos negativos). Para solventar esto se utiliza la función `collate_batch`.

Una alternativa a esto sería que el método get_item del dataset ya devolviera los datos en este formato. Por claridad, sin embargo, se optó por separar esto en la función indicada.

Un aspecto importante es que en los vocabularios generados no se incluye un "concepto" de relleno, necesario para esta función. Es por esto que se optó por usar como índice para este "pseudo-concepto" el tamaño del vocabulario (es decir, el índice del último concepto más 1).

In [178]:
def collate_batch(data):
  max_len = max(len(d["context"]) + len(d["negatives"]) for d in data)
  centers, contexts_negatives, masks, labels = [], [], [], []
  for d in data:
    center = d["center"]
    context = d["context"]
    negative = d["negatives"]
    centers += [center]
    cur_len = len(context) + len(negative)
    contexts_negatives += [context + negative + [len(vocabulary)] * (max_len - cur_len)]
    masks += [[1] * cur_len + [0] * (max_len - cur_len)]
    labels += [[1] * len(context) + [0] * (max_len - len(context))]
  return (torch.tensor(centers).reshape((-1, 1)), torch.tensor(
        contexts_negatives), torch.tensor(masks), torch.tensor(labels))

In [179]:
batch_size = 512
dataloader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, collate_fn=collate_batch)

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

Nótese una vez más la indicación de cuál concepto corresponde al relleno.

In [180]:
from torch import nn

class SkipGram(nn.Module):
    def __init__(self, vocabulary, embed_size):
        super().__init__()
        self.central_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))
        self.context_embedding = nn.Embedding(num_embeddings=len(vocabulary)+1,
                                embedding_dim=embed_size, padding_idx=len(vocabulary))

    def forward(self, center, contexts_and_negatives):
        v = self.central_embedding(center)
        u = self.context_embedding(contexts_and_negatives)
        pred = torch.bmm(v, u.permute(0, 2, 1))
        return pred

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionados.

In [181]:
class SigmoidBCELoss(nn.Module):
    # Binary cross-entropy loss with masking
    def __init__(self):
        super().__init__()

    def forward(self, inputs, target, mask=None):
        out = nn.functional.binary_cross_entropy_with_logits(
            inputs, target, weight=mask, reduction="none")
        return out.mean(dim=1)

loss = SigmoidBCELoss()

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

In [182]:
import time
def train(net, data_iter, lr, num_epochs, device):
    def init_weights(module):
        if type(module) == nn.Embedding:
            nn.init.xavier_uniform_(module.weight)
    net.apply(init_weights)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    L = 0
    N = 0
    for epoch in range(num_epochs):
        start, num_batches = time.time(), len(data_iter)
        for i, batch in enumerate(data_iter):
            optimizer.zero_grad()
            center, context_negative, mask, label = [
                data.to(device) for data in batch]

            pred = net(center, context_negative)
            l = (loss(pred.reshape(label.shape).float(), label.float(), mask)
                     / mask.sum(axis=1) * mask.shape[1])
            l.sum().backward()
            optimizer.step()
            L += l.sum()
            N += l.numel()
        print(f'loss {L / N:.3f}, '
          f'{N / (time.time() - start):.1f} tokens/sec on {str(device)}')

Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [183]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento. Esto se realizó con cada dataset, almacenando los parámetros resultantes de cada uno (variando tamaños de los embeddings y de lote) en /data/models.

La siguiente función permite entrenar con múltiples tamaños de embedding y de lote.

In [ ]:
def train_multiple(datasets_path, models_path, lr, num_epochs, embed_sizes, batch_sizes):
    datasets_files = get_filenames(datasets_path)

    pathlib.Path(models_path).mkdir(parents=True, exist_ok=True) 

    for f in datasets_files:
        ds = ISIDataset(datasets_path + "/" + f + ".pkl")
        for bs in batch_sizes:
            for es in embed_sizes:
                print("\033[94mEntrenando modelo " + f + "-" + str(bs) + "-" + str(es) + "\033[0m")
                dl = torch.utils.data.DataLoader(ds, bs, shuffle=True, collate_fn=collate_batch)
                isinet = SkipGram(vocabulary, es)
                train(isinet, dl, lr, num_epochs, try_gpu())
                torch.save(isinet.state_dict(), models_path + "/" + f + "-" + str(bs) + "-" + str(es) + ".pt")

In [ ]:
lr = 0.002
num_epochs = 10

embed_sizes = [256, 512]
batch_sizes = [256]

train_multiple(datasets_path, models_path, lr, num_epochs, embed_sizes, batch_sizes)

Entrenando modelo dataset-100-256-256
loss 0.355, 13760.1 tokens/sec on cpu
loss 0.334, 27285.5 tokens/sec on cpu
loss 0.326, 41083.0 tokens/sec on cpu
loss 0.321, 53759.6 tokens/sec on cpu
loss 0.318, 64441.9 tokens/sec on cpu
loss 0.316, 75401.6 tokens/sec on cpu
loss 0.314, 90294.8 tokens/sec on cpu
loss 0.313, 108025.6 tokens/sec on cpu
loss 0.312, 122884.8 tokens/sec on cpu
loss 0.311, 140116.1 tokens/sec on cpu
Entrenando modelo dataset-100-256-512


KeyboardInterrupt: 

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función, que devuelve los k conceptos cuyos embeddings se encuentran más cerca a cierto concepto:

In [189]:
def get_related_concepts(concept_ix, k, embed):
    W = embed.weight.data
    x = W[torch.tensor(concept_ix)]

    cos = torch.mv(W, x) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      torch.sum(x * x) + 1e-9)
    topk = torch.topk(cos, k=k+1)[1].cpu().numpy().astype('int32')

    related = []
    for i in topk[1:]:
        related.append(vocabulary[i])
    return related

In [191]:
isinet = SkipGram(vocabulary, 256)
isinet.load_state_dict(torch.load(models_path + '/dataset-100-256-256.pt',weights_only=True))

<All keys matched successfully>

Algunos ejemplos de la misma serían:

In [ ]:
# ('transmisión', 'datos')
get_related_concepts(80, 10, isinet.central_embedding)

[('datos', 'digitales'),
 ('modulación',),
 ('banda', 'voz'),
 ('velocidad', 'transmisión'),
 ('frecuencias',),
 ('comunicación', 'datos'),
 ('ruido',),
 ('errores',),
 ('señales',),
 ('circuito',)]

In [195]:
# ('segmento',)
get_related_concepts(72, 10, isinet.central_embedding)

[('número', 'secuencia'),
 ('entidad', 'transporte'),
 ('tamaño', 'ventana'),
 ('transporte',),
 ('bytes',),
 ('confirmación', 'recepción'),
 ('datagrama', 'ip'),
 ('número', 'puerto'),
 ('capa', 'transporte'),
 ('tamaño',)]

In [196]:
# ('control', 'congestión', 'tcp')
get_related_concepts(376, 10, isinet.central_embedding)

[('control', 'congestión'),
 ('tamaño', 'ventana'),
 ('datos', 'fiable'),
 ('conexión', 'tcp'),
 ('capa', 'transporte'),
 ('transferencia', 'datos', 'fiable'),
 ('tasa', 'transferencia'),
 ('transporte',),
 ('protocolo', 'transporte'),
 ('tcp', 'udp')]